In [1]:
import numpy as np
import copy
import pickle

from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti, save_nifti

from dipy.viz import regtools
from dipy.viz import actor, window, ui
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)

from dipy.tracking import streamline

import simnibs
from simnibs import sim_struct, run_simnibs

import vtkplotter

embedWindow(verbose=True): could not load k3d module, try:
> pip install k3d      # and if necessary:
> conda install nodejs


In [2]:
# reads the tractography data in trk format
# extracts streamlines and the file header. Streamlines should be in the same coordinate system as the FA map (used later).
# input example: '/home/sofya/RNF/NOVOKOV/Diffusion/DTI/CST_6.trk'
tractography_file=input("Please, specify the file with tracts that you would like to analyse. File should be in the trk format")

streams, hdr = load_trk(tractography_file)
streams_array=np.asarray(streams)
print ('imported tractography data:'+tractography_file)

Please, specify the file with tracts that you would like to analyse. File should be in the trk format/home/sofya/RNF/NOVOKOV/Diffusion/DTI/cst_1358.trk
imported tractography data:/home/sofya/RNF/NOVOKOV/Diffusion/DTI/cst_1358.trk


In [6]:
# load T1fs_conform image that operates in the same coordinates as simnibs except for the fact the center of mesh 
# is located at the image center
# T1fs_conform image should be generated in advance during the head meshing procedure
# input example: fname_T1='/home/sofya/RNF/NOVOKOV/T1/m2m_NOVIKOV/T1fs_conform.nii.gz'

fname_T1=input("Please, specify the T1fs_conform image that has been generated during head meshing procedure")
data_T1, affine_T1 = load_nifti(fname_T1)

# load FA image in the same coordinates as tracts
# input example:fname_FA='/home/sofya/RNF/NOVOKOV/Diffusion/DTI/dti_fa.nii'
fname_FA=input("Please, specify the FA image that has been generated during head meshing procedure")
data_FA, affine_FA = load_nifti(fname_FA)

print ('loaded T1fs_conform.nii and FA images')

Please, specify the T1fs_conform image that has been generated during head meshing procedure/home/sofya/RNF/NOVOKOV/T1/m2m_NOVIKOV/T1fs_conform.nii.gz
Please, specify the FA image that has been generated during head meshing procedure/home/sofya/RNF/NOVOKOV/Diffusion/DTI/dti_fa.nii
loaded T1fs_conform.nii and FA images


In [7]:
# specify the head mesh file that is used later in simnibs to simulate induced electric field
# input example:'/media/sofya/Seagate Backup Plus Drive/NOVOKOV/T1/NOVIKOV.msh'
mesh_path=input("Please, specify the head mesh file")

last_slach=max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
subject_name=mesh_path[last_slach:-4]

# specify the directory where you would like to save your simulation results
# input example:'/media/sofya/Seagate Backup Plus Drive/NOVOKOV/T1/StimVis'
out_dir=input("Please, specify the directory where you would like to save your simulation results")
out_dir=out_dir+'/simulation_at_pos_'

Please, specify the head mesh file/home/sofya/Example_data/subject.msh
Please, specify the directory where you would like to save your simulation results/home/sofya/Example_data/Output


In [8]:
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1,
                                      data_FA, affine_FA)
print ('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                              affine_T1, affine_FA,
                              starting_affine=starting_affine)
print ('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0,
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print ('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images. 
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)

print ('calculated Affine 3D transform')

calculated c_of_mass transformation
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated 3D translation transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Rigid 3D transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Affine 3D transform


In [9]:
identity = np.eye(4)

inv_affine_FA=np.linalg.inv(affine_FA)
inv_affine_T1=np.linalg.inv(affine_T1)
inv_affine=np.linalg.inv(affine.affine) 

# transforming streamlines to FA space
new_streams_FA=streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array=np.asarray(new_streams_FA)

T1_to_FA=np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1)) 
FA_to_T1=np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1=streamline.transform_streamlines(new_streams_FA, FA_to_T1)
new_streams_T1_array=np.asarray(new_streams_T1)

In [10]:
# # исключительно для визуализации и в DAMDID
all_new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
all_new_streams_T1_array = np.asarray(all_new_streams_T1)

new_streams_T1 = []
selected_fibers = [73, 170, 172, 208, 210, 211, 255, 261, 263, 700, 701, 773] 
for good_fiber in selected_fibers :
    new_streams_T1.append(all_new_streams_T1_array[good_fiber])

new_streams_T1_array = np.asarray(new_streams_T1)

In [11]:
# a function to compute 1st-order numerical derivative using a 3-point schema
# t - a point (index in the 'line' array) at which derivative should be computed
# line - array representing a function
# h - step between the points on the line

def my_deriv(t,line,h=1):
    if t==0:
        return (-3*line[t]+4*line[t+1]-line[t+2])/(2*h)
    elif t==len(line)-1:
        return (line[t-2]-4*line[t-1]+3*line[t])/(2*h)
    else:
        return (line[t+1]-line[t-1])/(2*h)

In [12]:
# calculating amline derivatives along the streamlines to get the local orientation of the streamlines

streams_array_derivative=copy.deepcopy(new_streams_T1_array)

print ('calculating amline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam=new_streams_T1_array[stream]
    for t in range(len(my_steam[:,0])):
        streams_array_derivative[stream][t,0]=my_deriv(t,my_steam[:,0])
        streams_array_derivative[stream][t,1]=my_deriv(t,my_steam[:,1])
        streams_array_derivative[stream][t,2]=my_deriv(t,my_steam[:,2])
        deriv_norm=np.linalg.norm(streams_array_derivative[stream][t,:])
        streams_array_derivative[stream][t,:]=streams_array_derivative[stream][t,:]/deriv_norm

calculating amline derivatives


In [13]:
# This function is to run simulations of the induced magnetic field using simnibs software

def simulation(fnamehead, pathfem, pos_centre=[-74.296158, -10.213354, 28.307243], pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'
    
    # Initialize a coil position
    pos = tmslist.add_position()
    pos.centre = pos_centre # Place the coil over
    pos.pos_ydir = pos_ydir # Point the coil towards
    pos.distance = distance # Distance between coil and head
    pos.didt = current_change # Rate of change of current in the coil, in A/s.
    run_simnibs(s)

In [61]:
list_streams_T1=list(new_streams_T1)
# adding one fictive bundle of length 1 with coordinates [0,0,0] to avoid some bugs with actor.line during visualization
list_streams_T1.append(np.array([0,0,0]))

bundle_native = list_streams_T1

# generating a list of colors to visualize later the stimualtion effects
effect_max=+1000000
effect_min=-1000000

In [14]:
#%%cython -a
def load_elems(nodes,elems):

    import numpy as np

    elems = elems[elems[:,3]!= -1,:]
    # Computing rectangles
    tmp = nodes[elems-1,:]
    elems_min = tmp.min(axis=1)
    elems_max = tmp.max(axis=1)
    tmp = 0
    sizes = (elems_max-elems_min).max(axis=0)
    # It is the index to reduce the elements to check
    order_min = np.argsort(elems_min[:,0])
    return {"Nodes":nodes, "Elems":elems, "El_min":elems_min, "El_max":elems_max, "Sizes":sizes,"Order_min":order_min}

def get_ttrd(loaded_elems,point):
    import numpy as np
    # Just to use names I have used before
    nodes = loaded_elems["Nodes"]
    elems = loaded_elems["Elems"]
    elems_min = loaded_elems["El_min"]
    elems_max = loaded_elems["El_max"]
    sizes = loaded_elems["Sizes"]
    order_min = loaded_elems["Order_min"]
    
    # Binary search to reduce the iterating points from 4mln to around 200k.
    r = np.searchsorted(elems_min[:,0],point[0],side='left',sorter=order_min)
    l = np.searchsorted(elems_min[:,0],point[0] - sizes[0],side='right',sorter=order_min)
    # Projection the data to only these points
    e_max = elems_max[order_min[l:r]]
    e_min = elems_min[order_min[l:r]]
    
    # Checks which ttrds are possible to contain the point
    potential_ttrds = order_min[l:r][(point[0] <= e_max[:,0]) & (e_min[:,1]<= point[1]) & (point[1] <= e_max[:,1]) & (e_min[:,2]<= point[2]) & (point[2] <= e_max[:,2])]
    
    # It checks if the ttrd contains the point
    def check_ttrd(ttrd, point):
        coord = np.column_stack((ttrd[1,:]-ttrd[0,:],ttrd[2,:]-ttrd[0,:],ttrd[3,:]-ttrd[0,:]))
        coord = np.linalg.inv(coord).dot(point-ttrd[0,:])
        return coord.min() >= 0 and coord.sum() <= 1
    # It checks if the ttrd with num ttrdNum contains the point
    def check_ttrd_byNum(ttrdNum, point):
        ttrd = nodes[elems[ttrdNum]-1]
        return check_ttrd(ttrd,point)
    
    # Just takes all ttrds that contain points
    nodeIndices = elems[[x for x in potential_ttrds if check_ttrd_byNum(x,point)]][0]; 
    ns = nodes[nodeIndices-1]

    norms = np.sum((ns-point)**2,axis=-1)**0.5
    weights = 1/(norms+1e-10)
    weights = weights / weights.sum()
    
    return {"Nodes":nodeIndices,"Weights":weights}

In [15]:
# a function to get e-field vector at a given position [x,y,z]
def get_field(ttt, point, my_field):
    best_ttt=get_ttrd(ttt,point)
    return np.dot(my_field[best_ttt['Nodes']-1].T, best_ttt['Weights'])

In [16]:
# a function to calculate directional derivatives of the effective field at a given point [x,y,z]
def deriv_e_field(coordinates, e_field_nodes, LSD, ttt):
   
    step=0.05

    x1=coordinates[0]
    y1=coordinates[1]
    z1=coordinates[2]
    x0=coordinates[0]-step
    x2=coordinates[0]+step
    y0=coordinates[1]-step
    y2=coordinates[1]+step
    z0=coordinates[2]-step
    z2=coordinates[2]+step

    f_x0_y1_z1=np.dot(get_field(ttt,np.asarray([x0,y1,z1]), e_field_nodes), LSD)
    f_x2_y1_z1=np.dot(get_field(ttt,np.asarray([x2,y1,z1]), e_field_nodes), LSD)
    f_x1_y1_z1=np.dot(get_field(ttt,np.asarray([x1,y1,z1]), e_field_nodes), LSD)
    f_x1_y0_z1=np.dot(get_field(ttt,np.asarray([x1,y0,z1]), e_field_nodes), LSD)
    f_x1_y2_z1=np.dot(get_field(ttt,np.asarray([x1,y2,z1]), e_field_nodes), LSD)
    f_x1_y1_z0=np.dot(get_field(ttt,np.asarray([x1,y1,z0]), e_field_nodes), LSD)
    f_x1_y1_z2=np.dot(get_field(ttt,np.asarray([x1,y1,z2]), e_field_nodes), LSD)
    
    gradx=my_deriv(1,[f_x0_y1_z1,f_x1_y1_z1,f_x2_y1_z1], step)
    grady=my_deriv(1,[f_x1_y0_z1,f_x1_y1_z1,f_x1_y2_z1], step)
    gradz=my_deriv(1,[f_x1_y1_z0,f_x1_y1_z1,f_x1_y1_z2], step)
    
    return np.dot([gradx, grady, gradz], LSD)

In [18]:
# a function to compute the TMS effects for a given coil position and coil direction
def change_TMS_effects(x,y,z, x_dir, y_dir, z_dir):
    
    l = 2 # membrane space constant 2mm
    l2 = l**2
    effect_max = -1000000

    position = [x - 256/2, y - 256/2, z - 256/2]
    direction = [x_dir - 256/2, y_dir - 256/2, z - 256/2]
    current_out_dir = out_dir + str(x) + '_' + str(y) + '_' + str(z)
    simulation(mesh_path,current_out_dir,pos_centre=position, pos_ydir=direction)
    
    mesh_file = current_out_dir+'/'+subject_name+'_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
    field_mesh = simnibs.msh.read_msh(mesh_file)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)
    
    effective_field=copy.deepcopy(new_streams_T1_array)
    
    for stream in range(len(new_streams_T1_array)):
        my_steam = copy.deepcopy(new_streams_T1_array[stream])
        print ('starting _' + str(stream) + ' out of ' + str(len(new_streams_T1_array)))
        for t in range(len(my_steam[:, 0])):
            #-256/2 because of a freesurfer RAS coordinate system
            x = my_steam[t, 0] - 256/2
            y = my_steam[t, 1] - 256/2
            z = my_steam[t, 2] - 256/2
            xyz=np.asarray([x, y, z])

            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            effective_field[stream][t,0] = l*np.dot(field_vector_xyz,streams_array_derivative[stream][t,:]) 
            effective_field[stream][t,1] = l2*deriv_e_field(xyz,field_at_nodes,streams_array_derivative[stream][t,:],ttt)
            effective_field[stream][t,2] = effective_field[stream][t,0]+effective_field[stream][t,1]
            if (effective_field[stream][t,2] > effect_max):
                effect_max=effective_field[stream][t,2]
            
    with open(current_out_dir + '/' + subject_name + '_effective_field_correct_12.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()

    return effect_max